In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
# install IGDB game database API
install_IGDB_API = False
if install_IGDB_API:
    !pip install igdb-api-v4

In [6]:
client_id = 'e3a6v5rouhozjbbcjticv7t63zg5o0'
client_secret = 'nkkhe32amgzw6bvd71upim8zdryl6c'

In [10]:
import requests
client_id = client_id
client_secret = client_secret

body = {
    'client_id': client_id,
    'client_secret': client_secret,
    'grant_type': 'client_credentials'
}

r = requests.post('https://id.twitch.tv/oauth2/token', body)

In [11]:
# data output
keys = r.json()

In [14]:
headers = {
    'Client-ID': client_id,
    'Authorization': 'Bearer ' + keys['access_token']
}

app_token = headers['Authorization']

In [68]:
"""IGDB wrapper module for the api v4 with APIcalypse"""

from requests import post
from requests.models import Request, Response

API_URL = "https://api.igdb.com/v4/"

class IGDBWrapper:
    def __init__(self, client_id:str, auth_token:str) -> None:
        self.client_id = client_id
        self.auth_token = auth_token

    def api_request(self, endpoint:str, query:str) -> Response:
        """
        Takes an endpoint and the APIcalypse query and returns the api response as a byte string.
        """
        url = IGDBWrapper._build_url(endpoint)
        params = self._compose_request(query)

        response = post(url, **params)
        response.raise_for_status()

        return response

    @staticmethod
    def _build_url(endpoint:str='') -> str:
        return ('%s%s' % (API_URL, endpoint))

    def _compose_request(self, query:str) -> Request:
        if not query:
            raise Exception('No query provided!\nEither provide an inline query following Apicalypse\'s syntax or an Apicalypse object')
       
        request_params = {
            'headers': {
                'Client-ID': self.client_id,
                'Authorization': ('Bearer %s' % (self.auth_token)),
            }
        }

        if isinstance(query, str):
            request_params['data'] = query
            return request_params

        raise TypeError('Incorrect type of argument \'query\', only Apicalypse-like strings or Apicalypse objects are allowed')

In [69]:
# from igdb.wrapper import IGDBWrapper
wrapper = IGDBWrapper(headers['Client-ID'], keys['access_token'])

In [348]:
# JSON API request
# (48, 46) API code for PS4 or XBOX ONE

def get_game_data(page, page_elements):
    byte_array = wrapper.api_request(
                'games',
                f'fields name, category, first_release_date, summary; offset {page*page_elements}; limit 500; where (release_dates.platform = (48, 49) & release_dates.y >= 2014);;'
              )
    game_data = byte_array.json()
    return game_data

In [349]:
list_of_games = []

flag = 1
page = 0

while flag == 1:
    game_data = get_game_data(page=page, page_elements=10)
    list_of_games.extend(game_data)
    if not game_data:
        flag = 2
    page += 1

In [350]:
len(list_of_games)

422100

In [352]:
columns=['id', 'name', 'category', 'summary']

ids = []
names = []

for game in list_of_games:
    if 'summary' in game.keys() and game['category'] == 0:
        ids.append(game['id'])
        names.append(game['name'])
        summaries.append(game['summary'])

In [356]:
len(ids)

299026

In [359]:
game_dict = {'id': ids, 'name': names, 'summary': summaries}
game_df = pd.DataFrame(game_dict)

In [360]:
game_df.head(15)

,id,name,summary
0,4,Thief,There is a rising tide of fear in The City. Ha...
1,18,Max Payne,"Max Payne, gritty ex-cop and titular hero of t..."
2,79,Stubbs the Zombie in Rebel Without a Pulse,The game is set in a fictional City of the Fut...
3,83,Baldur's Gate: Dark Alliance,Baldur’s Gate™: Dark Alliance™ is a revolution...
4,111,Amnesia: The Dark Descent,Amnesia: The Dark Descent is a survival horror...
5,117,The Last Guardian,"In a strange and mystical land, a young boy di..."
6,120,Diablo III,"The game takes place in Sanctuary, the dark fa..."
7,134,Devil May Cry,Legend has it that 2000 years ago in the depth...
8,135,Devil May Cry 2,"Dante is back! Prepare for the next, stylish c..."
9,148,Star Wars: Republic Commando,Chaos has erupted throughout the galaxy. As a ...


In [361]:
# remove duplicates
new_game_df = game_df.drop_duplicates(subset='name')

In [373]:
new_game_df[new_game_df['summary'].str.contains("Kratos")]

,id,name,summary
42345,19560,God of War,"""It is a new beginning for Kratos. Living as a..."
146886,83778,God of War: Stone Mason's Edition,"Stone Mason Edition Includes:\n\n9"" Kratos & A..."
147229,83824,God of War: Collector's Edition,"The game was released worldwide on April 20, 2..."
185975,112875,God of War: Ragnarok,God of War: Ragnarök is the tentative title fo...
192140,115067,God of War: Limited Edition,Limited Edition contains:\n- Steelbook Case\n-...


In [374]:
new_game_df[new_game_df['summary'].str.contains("Ellie")]

,id,name,summary
68112,26192,The Last of Us Part II,Set 5 years after the events of The Last of Us...
230847,130577,Little Witch in the Woods,"In the 'LUCEREIN ORTU', witches live together...."
240603,135955,The Last of Us Part II: Collector's Edition,Set 5 years after the events of The Last of Us...


In [375]:
new_game_df[new_game_df['name'].str.contains("Batman")]

,id,name,summary
237,5503,Batman: Arkham Knight,Batman: Arkham Knight brings the award-winning...
372,7685,LEGO Batman 3: Beyond Gotham,"In LEGO Batman 3: Beyond Gotham, the Caped Cru..."
24833,14746,Batman: The Telltale Series,Enter the fractured psyche of Bruce Wayne and ...
42348,19563,Batman: Arkham VR,Your quest to clean up Gotham has only just be...
111498,51525,Batman: The Enemy Within,"In this latest chapter, both Bruce Wayne and B..."
288153,164684,LEGO Batman 3: Beyond Gotham - Deluxe Edition,The Caped Crusader joins forces with the super...


In [365]:
new_game_df.to_csv('new_game_data.csv')

In [366]:
game_df = pd.read_csv('new_game_data.csv')
game_df = game_df.drop(columns='Unnamed: 0')

In [367]:
game_df.head()

,id,name,summary
0,4,Thief,There is a rising tide of fear in The City. Ha...
1,18,Max Payne,"Max Payne, gritty ex-cop and titular hero of t..."
2,79,Stubbs the Zombie in Rebel Without a Pulse,The game is set in a fictional City of the Fut...
3,83,Baldur's Gate: Dark Alliance,Baldur’s Gate™: Dark Alliance™ is a revolution...
4,111,Amnesia: The Dark Descent,Amnesia: The Dark Descent is a survival horror...


## Content Based Recommender System

In [376]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [377]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(game_df['summary'])

In [378]:
tfidf_matrix

<6197x428146 sparse matrix of type '<class 'numpy.float64'>'
	with 690778 stored elements in Compressed Sparse Row format>

In [379]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}

for idx, row in game_df.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], game_df['id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]

In [380]:
def item_name(id):
    return game_df.loc[game_df['id'] == id]['name']

In [381]:
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item_name(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print(f"Recommended: {item_name(rec[1])}")

In [384]:
recommend(item_id=19560, num=20)

1276    Recommending 20 products similar to God of War...
Name: name, dtype: object
-------
Recommended: 4230    God of War: Limited Edition
Name: name, dtype: object
Recommended: 3329    God of War: Stone Mason's Edition
Name: name, dtype: object
Recommended: 3332    God of War: Collector's Edition
Name: name, dtype: object
Recommended: 4113    God of War: Ragnarok
Name: name, dtype: object
Recommended: 1123    Jotun: Valhalla Edition
Name: name, dtype: object
Recommended: 1628    Kingdom: New Lands
Name: name, dtype: object
Recommended: 2491    Die for Valhalla!
Name: name, dtype: object
Recommended: 3052    The Colonists
Name: name, dtype: object
Recommended: 2738    Shadow Warrior Collection
Name: name, dtype: object
Recommended: 5993    Gods Will Fall: Valiant Edition
Name: name, dtype: object
Recommended: 336    Mass Effect: Andromeda
Name: name, dtype: object
Recommended: 5419    Immortals Fenyx Rising: Limited Edition
Name: name, dtype: object
Recommended: 3317    Outbreak Bund